# Job Generation

In [ ]:
import json
from pathlib import Path

from fl_manager.core.meta_registry import MetaRegistry
from fl_manager.core.executors import FLExecutorRegistry
from fl_manager.core.schemas.registry_item import RegistryItem
from fl_manager.core.schemas.fl_config import FLConfig
from fl_manager.utils.job_generator.job_creator import JobCreator
from fl_manager.utils.job_generator.schemas.job_config import JobConfig
from fl_manager.utils.job_generator.federated_job.pt import PTFederatedJobRegistry

In [ ]:
a = FLConfig(**json.loads([
  e for e in Path("../examples/nvflare_mnist/sag_pt__mnist_example/app/custom").glob('*.json')
][0].read_text()))

In [ ]:
q = a.fl_executor.keyword_arguments.copy()
components = q.pop('components')
fl_algorithm = q.pop('fl_algorithm')
fl_algorithm_kwargs = q.pop('fl_algorithm_kwargs')

### MNIST Example

In [ ]:
_components = components | {
  'dataset_distributor': RegistryItem(**{'registry_id': 'dataset_distributor', 'name': 'dirichlet',
                                         'keyword_arguments': {'target_col': 'label', 'num_clients': 10, 'alpha': 0.5,
                                                               'balancing': False, 'shuffle': False}}),
  'dataset_splitter': RegistryItem(**{'registry_id': 'dataset_splitter', 'name': 'stratified',
                                      'keyword_arguments': {'target_col': 'label', 'min_samples_per_class': 20}}),
  'fl_model': RegistryItem(**{'registry_id': 'fl_model', 'name': 'mnist_mobile_net',
                              'keyword_arguments': {'task': 'supervised-multiclass-image-classification',
                                                    'num_classes': 10, 'in_channels': 1}})
}
_components

FedAvg Job

In [ ]:
executor_name = a.fl_executor.name
jconfig = JobConfig(
  name='autogen_fedavg',
  num_rounds=3,
  executor=executor_name,
  executor_kwargs=q,
  runner='base',
  fl_algorithm={'name': 'fed_avg', 'client_kwargs': {}, 'server_kwargs': {}},
  components=_components,
  clients={'site-1': {}, 'site-2': {}}
)
JobCreator(jconfig).create("/opt/project/notebooks/jobs")

FedProx Job

In [ ]:
jconfig = JobConfig(
  name='autogen_fedprox',
  num_rounds=3,
  executor=a.fl_executor.name,
  executor_kwargs=q,
  runner='base',
  fl_algorithm={'name': 'fed_prox', 'client_kwargs': {'mu': 0.1}, 'server_kwargs': {}},
  components=_components,
  clients={'site-1': {}, 'site-2': {}}
)
JobCreator(jconfig).create("/opt/project/notebooks/jobs")

FedOpt Job

In [ ]:
jconfig = JobConfig(
  name='autogen_fedopt',
  num_rounds=3,
  executor=a.fl_executor.name,
  executor_kwargs=q,
  runner='base',
  fl_algorithm={'name': 'fed_opt', 'client_kwargs': {}, 'server_kwargs': {}},
  components=_components,
  clients={'site-1': {}, 'site-2': {}}
)
JobCreator(jconfig).create("/opt/project/notebooks/jobs")

FedBN Job

In [ ]:
jconfig = JobConfig(
  name='autogen_fedbn',
  num_rounds=3,
  executor=executor_name,
  executor_kwargs=q,
  runner='base',
  fl_algorithm={'name': 'fed_bn', 'client_kwargs': {}, 'server_kwargs': {}},
  components=_components,
  clients={'site-1': {}, 'site-2': {}}
)
JobCreator(jconfig).create("/opt/project/notebooks/jobs")

### NLP Example

In [ ]:
_nlp_components = components | {
  'dataset_reader': RegistryItem(**{"registry_id": "dataset_reader", "name": "huggingface",
                                    "keyword_arguments": {"dataset_name": "fancyzhx/ag_news",
                                                          "dataset_mapping": {"train": "train", "test": "test"}}}),
  'dataset_splitter': RegistryItem(**{"registry_id": "dataset_splitter", "name": "stratified",
                                      "keyword_arguments": {"target_col": "label", "min_samples_per_class": 20}}),
  'dataset_distributor': RegistryItem(**{'registry_id': 'dataset_distributor', 'name': 'dirichlet',
                                         'keyword_arguments': {'target_col': 'label', 'num_clients': 50,
                                                               'alpha': 3.5}}),
  'dataframe_dataset': RegistryItem(**{"registry_id": "dataframe_dataset", "name": "torch_dataframe_transforms_dataset",
                                       "keyword_arguments": {"output_keys": ["text", "label"], "transforms": {}}}),
  'fl_model': RegistryItem(**{'registry_id': 'fl_model', 'name': 'tiny_bert_classifier',
                              'keyword_arguments': {'task': 'supervised-multiclass-text-classification',
                                                    'num_classes': 4}})
}
_nlp_components.pop("dataset_formatter")
_nlp_components.pop("dataset_validator")
_nlp_components

In [ ]:
_q = q | {
  'datamodule_name': 'tokenizer',
  'datamodule_kwargs': {"batch_size": 64, "num_workers": 8, "max_length": 128,
                        "tokenizer_name": "huawei-noah/TinyBERT_General_4L_312D"}
}
_q

FedAvg Job

In [ ]:
jconfig = JobConfig(
  name='autogen_nlp_fedavg',
  num_rounds=3,
  executor=a.fl_executor.name,
  executor_kwargs=_q,
  runner='base',
  fl_algorithm={'name': 'fed_avg', 'client_kwargs': {}, 'server_kwargs': {}},
  components=_nlp_components,
  clients={'site-1': {}, 'site-2': {}}
)
JobCreator(jconfig).create("/opt/project/notebooks/jobs")